In [ ]:
import pandas as pd
import numpy as np
import warp_norm
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
data = pd.read_csv('/home/hgh/hghData/Datasets/preprocessed_labels.csv')
image_path = data.loc[:, 'image_path']
label = data.loc[:, 'label']
results = np.loadtxt('./results.txt')

In [ ]:
img = cv2.imread(image_path[105])
print(label[105])
cleaned_string = label[105].replace('\n', '').replace(' ', '')
# 将字符串切分成单个数字的列表
numbers = cleaned_string.split('][')
numbers[0] = numbers[0].lstrip('[')
numbers[-1] = numbers[-1].rstrip(']')
# 将字符串转换为浮点数，并重新组成 NumPy 数组
numpy_array = np.array([float(num) for num in numbers]).reshape(3,1)
gaze_img = warp_norm.draw_gaze(img, numpy_array, color = (0,255,0))
gaze_img = warp_norm.draw_gaze(img, results[105])
plt.imshow(cv2.cvtColor(gaze_img, cv2.COLOR_BGR2RGB))
plt.show()

In [1]:
import os
import pandas as pd
import numpy as np
import pickle

csv_file_path = '/home/hgh/hghData/Datasets/preprocessed_labels.csv'
df = pd.read_csv(csv_file_path)
dataset_dict = {}

for index, row in df.iterrows():
    image_path = row['image_path']
    label = row['label']
    cleaned_string = label.replace('\n', '').replace(' ', '')
    # 将字符串切分成单个数字的列表
    numbers = cleaned_string.split('][')
    numbers[0] = numbers[0].lstrip('[')
    numbers[-1] = numbers[-1].rstrip(']')  
    # 组织样本信息为字典
    label_array = np.array([float(num) for num in numbers]).reshape(3,1)
    sample_info = {
        'image_path': image_path,
        'label': label_array
    }

    # 将样本信息加入数据集字典，以索引作为键
    dataset_dict[index] = sample_info

# for key, value in dataset_dict.items():
#     print(f"Sample {key}: {value}")

pickle_file_path = '/home/hgh/hghData/Datasets/dataset_dict.pkl'
with open(pickle_file_path, 'wb') as file:
    pickle.dump(dataset_dict, file)


In [36]:
import warp_norm
import numpy as np
gv = np.array([0.0685,0.3065,-0.94944])
if gv.size == 2:
    gv = warp_norm.pitchyaw_to_vector(gv)
z = np.array([0,0,-600])
theta = np.arcsin(np.linalg.norm(np.cross(gv,z))/(np.linalg.norm(gv)*np.linalg.norm(z)))
print(theta)
scale = np.linalg.norm(z)/(np.cos(theta)*np.linalg.norm(gv))
print(scale)
gp = scale * gv - z
print(gp)
gp = np.delete(gp, 2, axis=0)
t = np.array([0.215,0.215])
s = np.array([500,500])
print(gp/t+s)

0.31945605309159664
631.9514661274015
[ 4.32886754e+01  1.93693124e+02 -1.13686838e-13]
[ 701.34267642 1400.89825287]


In [19]:
import os
import pandas as pd
import numpy as np
import pickle
import warp_norm
import utils

pickle_file_path = '/home/hgh/hghData/Datasets/dataset_dict.pkl'
with open(pickle_file_path, 'rb') as file:
    loaded_dataset_dict = pickle.load(file)
ground_truth = []
idx = 0
tan_list = []
for key, value in loaded_dataset_dict.items():
    # print(f"Sample {key}: {value}")
    # 取出男生的数据
    image_name = os.path.basename(value['image_path'])
    # print(image_name)
    number = int((int(os.path.splitext(image_name)[0][19:]) - 1)/100)
    if number % 2 == 0:
        tan_list.append(idx)
    ground_truth.append(value['label'].reshape((1,3))[0])
    idx+=1
ground_truth = np.array(ground_truth)
print(tan_list)
ground_truth = warp_norm.vector_to_pitchyaw(-ground_truth)
print(ground_truth)

pred = []
pred_path = './results.txt'

with open(pred_path, 'r') as file:
    lines = file.readlines()

for line in lines:
    
    data_array = [float(x) for x in line.split()]
    pred.append(data_array)
pred = np.array(pred)
print(pred)

ground_truth_tan = ground_truth[tan_list]
pred_tan = pred[tan_list]

mask = np.isin(np.arange(len(ground_truth)), tan_list, invert=True)
ground_truth_chen = ground_truth[mask]
pred_chen = pred[mask]


e_tan = utils.angular_error(ground_truth_tan,pred_tan)
print('Tan error:', np.mean(e_tan))


e_chen = utils.angular_error(ground_truth_chen,pred_chen)
print('Chen error:', np.mean(e_chen))

e = utils.angular_error(ground_truth,pred)
print('Total error:', np.mean(e))



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418,